<a href="https://colab.research.google.com/github/vhrique/ELF/blob/main/20250211_Analise_Sucupira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
planilhas = {
    '2021': 'https://drive.google.com/uc?export=download&id=1aY-35G2D8oFIwY9UtxaUP9K00Cz72Tf9',
    '2022': 'https://drive.google.com/uc?export=download&id=14QUdOCIMWLOL3De60GKPkvlgxzaMCDPz',
    '2023': 'https://drive.google.com/uc?export=download&id=1C2qkNL1N9B6Q6wUh33kbkQ8QRySnEfnZ',
    # '2024': 'https://drive.google.com/uc?export=download&id=1WLvc4wt8RsiN2Ob6UWjGq9SodFvQORCj',
}

In [3]:
df = {}
for ano in planilhas:
    df[ano] = pd.read_excel(planilhas[ano], sheet_name=None)

In [15]:
dfs = []
for ano in df:
    dfs.append(df[ano]['Linhas de Pesquisa - Projetos'])
df_projeto_linhas = pd.concat(dfs).reset_index(drop=True)

In [16]:
dfs = []
for ano in df:
    dfs.append(df[ano]['Projetos - Membros'])
df_projeto_membros = pd.concat(dfs).reset_index(drop=True)

In [69]:
docentes = df_projeto_membros.loc[df_projeto_membros['Categoria do Membro do Projeto'] == 'Docente', 'Nome do Membro do Projeto'].unique().tolist()

In [5]:
dfs = []
for ano in df:
    dfs.append(df[ano]['Projetos - Financiadores'])
df_projeto_financiadores = pd.concat(dfs).reset_index(drop=True)

In [ ]:
df = df_projeto_linhas.merge(df_projeto_membros)

In [6]:
projetos_financiados = df_projeto_financiadores['Nome do Projeto de Pesquisa'].unique().tolist()

In [67]:
df_projeto_membros['Categoria do Membro do Projeto']

,Categoria do Membro do Projeto
0,Discente
1,Docente
2,Docente
3,Docente
4,Docente
...,...
969,Egresso
970,Participante Externo
971,Docente
972,Docente


In [7]:
projetos_com_membros = df_projeto_membros['Nome do Projeto de Pesquisa'].unique().tolist()

In [8]:
projetos_todos = list(set(projetos_financiados + projetos_com_membros))

In [29]:
CDIAC = []
GOP = []
IDP = []
IAS = []

for projeto in projetos_todos:
    df_projeto = df_projeto_linhas.loc[df_projeto_linhas['Nome do Projeto de Pesquisa'] == projeto]
    CDIAC.append('CIÊNCIA DE DADOS, INTELIGÊNCIA ARTIFICIAL E CONTROLE' in df_projeto['Linha de Pesquisa'].tolist())
    GOP.append('GESTÃO DE OPERAÇÕES' in df_projeto['Linha de Pesquisa'].tolist())
    IDP.append('INOVAÇÃO E DESENVOLVIMENTO DE PRODUTO' in df_projeto['Linha de Pesquisa'].tolist())
    IAS.append('INTEGRAÇÃO E AUTOMAÇÃO DE SISTEMAS' in df_projeto['Linha de Pesquisa'].tolist())

In [31]:
discente = []

for projeto in projetos_todos:
    df_projeto = df_projeto_membros.loc[df_projeto_membros['Nome do Projeto de Pesquisa'] == projeto]
    discente.append('Discente' in df_projeto['Categoria do Membro do Projeto'].tolist())

In [47]:
bolsa = []
auxilio = []

for projeto in projetos_todos:
    df_projeto = df_projeto_financiadores.loc[df_projeto_financiadores['Nome do Projeto de Pesquisa'] == projeto]
    bolsa.append('BOLSA' in df_projeto['Natureza do Financiador'].tolist())
    auxilio.append('OUTRO AUXÍLIO FINANCEIRO' in df_projeto['Natureza do Financiador'].tolist())

In [48]:
df = pd.DataFrame({
    'projeto': projetos_todos,
    'lp_cdiac': CDIAC,
    'lp_gop': GOP,
    'lp_idp': IDP,
    'lp_ias': IAS,
    'discente': discente,
    'bolsa': bolsa,
    'auxilio': auxilio
})
df['lp'] = df.iloc[:,1:-2].any(axis=1)

In [61]:
criterio_g1 = df.query('auxilio == True and bolsa == False').lp.mean()
criterio_g1

1.0

In [62]:
criterio_g2 = df.query('auxilio == True and bolsa == False').discente.mean()
criterio_g2

0.6666666666666666

In [71]:
# criterio_g3 = df.query('auxilio == True and bolsa == False').discente.mean()
# criterio_g3

In [64]:
criterio_g4 = (df.query('auxilio == True and bolsa == False').sum(axis=0).iloc[1:-2] > 0).mean()
criterio_g4

0.5